In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
import shap
from datetime import datetime

# Setup logging
logging.basicConfig(
    filename=f'feature_importance_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class FeatureImportanceAnalyzer:
    def __init__(self, continuous_features, discrete_features):
        """
        Initialize the analyzer with feature definitions.
        
        Args:
            continuous_features (list): List of continuous feature names
            discrete_features (list): List of discrete feature names
        """
        self.continuous_features = continuous_features
        self.discrete_features = discrete_features
        self.models = {
            'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
            'xgboost': xgb.XGBClassifier(n_estimators=100, random_state=42),
            'lightgbm': lgb.LGBMClassifier(n_estimators=100, random_state=42),
            'gradient_boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
            'logistic': LogisticRegression(max_iter=1000, random_state=42)
        }
        
    def prepare_data(self, df, window_size):
        """
        Prepare data for analysis with proper scaling and encoding.
        
        Args:
            df (pd.DataFrame): Input dataframe
            window_size (int): Size of the window for feature extraction
            
        Returns:
            tuple: (X, y, feature_names)
        """
        X_processed = []
        feature_names = []
        
        # Calculate middle position based on sequence length
        seq_length = len(df[self.continuous_features[0]].iloc[0])
        middle_pos = seq_length // 2
        
        # Validate window size
        if window_size * 2 + 1 > seq_length:
            raise ValueError(f"Window size {window_size} exceeds sequence length {seq_length}")
        
        # Process continuous features with position-based weighting
        scaler = StandardScaler()
        weights = np.exp(-np.abs(np.arange(-window_size, window_size+1)) / window_size) if window_size > 0 else [1]
        
        for feature in self.continuous_features:
            positions = list(range(middle_pos-window_size, middle_pos+window_size+1))
            for idx, pos in enumerate(positions):
                values = np.array([x[pos] for x in df[feature].values])
                mask = values != 0
                if mask.any():
                    non_zero_values = values[mask].reshape(-1, 1)
                    values[mask] = scaler.fit_transform(non_zero_values).flatten() * weights[idx]
                X_processed.append(values)
                feature_names.append(f"{feature}_{pos-middle_pos}")
        
        # Process SS with improved encoding
        encoder = OneHotEncoder(sparse_output=False)
        positions = list(range(middle_pos-window_size, middle_pos+window_size+1))
        for idx, pos in enumerate(positions):
            ss_values = np.array([x[pos] for x in df['ss'].values]).reshape(-1, 1)
            encoded_ss = encoder.fit_transform(ss_values)
            for i, ss_type in enumerate(encoder.categories_[0]):
                X_processed.append(encoded_ss[:, i] * weights[idx])
                feature_names.append(f"ss_{ss_type}_{pos-middle_pos}")
        
        X = np.array(X_processed).T
        y = df['label'].values
        
        return X, y, feature_names
    
    def get_model_importance(self, model_name, X, y, feature_names):
        """
        Get feature importance from a specific model with cross-validation.
        """
        model = self.models[model_name]
        
        # Perform cross-validation
        cv_scores = cross_val_score(model, X, y, cv=5)
        logging.info(f"{model_name} CV Score: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
        
        # Fit model and get importance
        model.fit(X, y)
        
        if model_name == 'logistic':
            importance = np.abs(model.coef_[0])
        else:
            importance = model.feature_importances_
            
        return importance
    
    def get_shap_importance(self, X, y, feature_names):
        """
        Calculate SHAP importance values.
        """
        model = xgb.XGBClassifier(n_estimators=100, random_state=42)
        model.fit(X, y)
        
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        return np.abs(shap_values).mean(axis=0)
    
    def analyze_feature_correlations(self, X, feature_names):
        """
        Analyze correlations between features.
        """
        corr_matrix = pd.DataFrame(X, columns=feature_names).corr()
        return corr_matrix
    
    def analyze_all_models(self, df, window_size):
        """
        Analyze feature importance using all models.
        """
        X, y, feature_names = self.prepare_data(df, window_size)
        
        results = {}
        # Get importance from each model
        for model_name in self.models.keys():
            try:
                importance = self.get_model_importance(model_name, X, y, feature_names)
                results[model_name] = self.aggregate_importance(importance, feature_names)
            except Exception as e:
                logging.error(f"Error in {model_name} analysis: {e}")
                continue
            
        # Add SHAP importance
        try:
            shap_importance = self.get_shap_importance(X, y, feature_names)
            results['shap'] = self.aggregate_importance(shap_importance, feature_names)
        except Exception as e:
            logging.error(f"SHAP analysis failed: {e}")
            
        # Add correlation analysis
        try:
            corr_matrix = self.analyze_feature_correlations(X, feature_names)
            results['correlations'] = corr_matrix
        except Exception as e:
            logging.error(f"Correlation analysis failed: {e}")
        
        return results
    
    def aggregate_importance(self, importance, feature_names):
        """
        Aggregate feature importance with improved handling.
        """
        feature_importance = {}
        for feat_type in self.continuous_features + ['ss']:
            if feat_type == 'ss':
                mask = [f.startswith('ss_') for f in feature_names]
            else:
                mask = [f.startswith(feat_type + '_') for f in feature_names]
            importance_values = importance[mask]
            
            feature_importance[feat_type] = {
                'mean': np.mean(importance_values),
                'std': np.std(importance_values),
                'max': np.max(importance_values),
                'min': np.min(importance_values)
            }
        return feature_importance

def print_model_importance_tables(all_results):
    """Print comprehensive feature importance tables for each model."""
    model_names = list(all_results[0].keys())
    window_sizes = list(all_results.keys())
    
    for model in model_names:
        if model == 'correlations':
            continue
            
        print(f"\n=== {model.upper()} ===")
        print("-" * 100)
        
        # Print header
        print(f"{'Feature':<15} ", end="")
        for size in window_sizes:
            print(f"Win±{size:<15} ", end="")
        print("\n" + "-" * 100)
        
        # Get all features
        features = list(all_results[0][model].keys())
        
        # Print each feature's importance
        for feature in sorted(features):
            print(f"{feature:<15} ", end="")
            for size in window_sizes:
                if model in all_results[size]:
                    importance = all_results[size][model].get(feature, {}).get('mean', 0)
                    std = all_results[size][model].get(feature, {}).get('std', 0)
                    print(f"{importance:,.4f}±{std:,.4f} ", end="")
                else:
                    print("N/A            ", end="")
            print()
        print("-" * 100)

def analyze_features(train_df_path):
    """
    Main function to analyze feature importance.
    """
    # Define features
    continuous_features = [
        'phi', 'psi', 'omega', 'tau', 
        'chi1', 'chi2', 'chi3', 'chi4',
        'sasa', 'plDDT'
    ]
    discrete_features = ['ss']
    
    # Initialize analyzer
    analyzer = FeatureImportanceAnalyzer(continuous_features, discrete_features)
    
    # Read and process data
    train_df_path = os.path.abspath(os.path.expanduser(train_df_path))
    logging.info(f"Reading data from: {train_df_path}")
    
    try:
        train_df = pd.read_csv(train_df_path)
    except Exception as e:
        logging.error(f"Error reading data: {e}")
        raise
    
    # Process features
    processed_df = pd.DataFrame()
    processed_df['label'] = train_df['label']
    
    def process_feature(df, feature_name):
        if feature_name == 'ss':
            return df[feature_name].apply(list)
        else:
            return df[feature_name].apply(lambda x: 
                np.array([float(v.strip()) for v in x.strip('[]').split(',')]))
    
    for feature in continuous_features:
        processed_df[feature] = process_feature(train_df, feature)
    processed_df['ss'] = train_df['ss'].apply(list)
    
    # Analyze with different window sizes
    window_sizes = [0, 1, 2, 4, 8, 16]
    all_results = {}
    
    for size in window_sizes:
        logging.info(f"Processing window size ±{size}")
        try:
            results = analyzer.analyze_all_models(processed_df, size)
            all_results[size] = results
        except Exception as e:
            logging.error(f"Error processing window size {size}: {e}")
            continue
    
    # Create results directory
    results_dir = 'feature_importance_results'
    os.makedirs(results_dir, exist_ok=True)
    
    # Save results and create visualizations
    save_results(all_results, results_dir, window_sizes)
    
    # Print analysis
    print("\n=== FEATURE IMPORTANCE ANALYSIS ===")
    print_model_importance_tables(all_results)
    
    return all_results

def save_results(all_results, results_dir, window_sizes):
    """
    Save analysis results and create visualizations.
    """
    for size in window_sizes:
        if 'correlations' in all_results[size]:
            corr_matrix = all_results[size]['correlations']
            plt.figure(figsize=(15, 12))
            sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm')
            plt.title(f'Feature Correlations (Window Size ±{size})')
            plt.tight_layout()
            plt.savefig(os.path.join(results_dir, f'correlation_matrix_win{size}.png'))
            plt.close()
            
            # Save correlation matrix
            corr_matrix.to_csv(os.path.join(results_dir, f'correlation_matrix_win{size}.csv'))
    
    # Save importance results for each model
    model_names = [name for name in all_results[0].keys() if name != 'correlations']
    
    for model_name in model_names:
        try:
            results_df = pd.DataFrame({
                size: {k: v['mean'] for k, v in results[model_name].items()}
                for size, results in all_results.items()
            })
            
            # Save numerical results
            results_df.to_csv(os.path.join(results_dir, f'feature_importance_{model_name}.csv'))
            
            # Create heatmap
            plt.figure(figsize=(12, 8))
            sns.heatmap(results_df, annot=True, fmt='.3f', cmap='YlOrRd')
            plt.title(f'Feature Importance vs Window Size ({model_name.upper()})')
            plt.xlabel('Window Size')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig(os.path.join(results_dir, f'feature_importance_heatmap_{model_name}.png'))
            plt.close()
            
        except Exception as e:
            logging.error(f"Error saving results for {model_name}: {e}")
            continue

if __name__ == "__main__":
    try:
        results = analyze_features('../../data/train/structure/processed_features_train.csv')
    except Exception as e:
        logging.error(f"Analysis failed: {e}")
        raise

/home/ubuntu/data/miniconda3/envs/finetune-dephos/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/ubuntu/data/miniconda3/envs/finetune-dephos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[LightGBM] [Info] Number of positive: 3673, number of negative: 3409
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000356 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2556
[LightGBM] [Info] Number of data points in the train set: 7082, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518639 -> initscore=0.074590
[LightGBM] [Info] Start training from score 0.074590
[LightGBM] [Info] Number of positive: 3673, number of negative: 3409
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2556
[LightGBM] [Info] Number of data points in the train set: 7082, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518639 -> initscore=0.074590
[LightGBM] [Info] Start training from score 0.074590
[LightGBM] [Info] Numb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
import shap

class FeatureImportanceAnalyzer:
    def __init__(self, continuous_features, discrete_features):
        self.continuous_features = continuous_features
        self.discrete_features = discrete_features
        self.models = {
            'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
            'xgboost': xgb.XGBClassifier(n_estimators=100, random_state=42),
            'lightgbm': lgb.LGBMClassifier(n_estimators=100, random_state=42),
            'gradient_boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
            'logistic': LogisticRegression(max_iter=1000, random_state=42)
        }
        
    def prepare_data(self, df, window_size, middle_pos=16):
        X_processed = []
        feature_names = []
        
        # Process continuous features
        scaler = StandardScaler()
        for feature in self.continuous_features:
            positions = list(range(middle_pos-window_size, middle_pos+window_size+1))
            for pos in positions:
                values = np.array([x[pos] for x in df[feature].values])
                mask = values != 0
                if mask.any():
                    non_zero_values = values[mask].reshape(-1, 1)
                    values[mask] = scaler.fit_transform(non_zero_values).flatten()
                X_processed.append(values)
                feature_names.append(f"{feature}_{pos-middle_pos}")
        
        # Process SS
        encoder = OneHotEncoder(sparse_output=False)
        positions = list(range(middle_pos-window_size, middle_pos+window_size+1))
        for pos in positions:
            ss_values = np.array([x[pos] for x in df['ss'].values]).reshape(-1, 1)
            encoded_ss = encoder.fit_transform(ss_values)
            for i, ss_type in enumerate(encoder.categories_[0]):
                X_processed.append(encoded_ss[:, i])
                feature_names.append(f"ss_{ss_type}_{pos-middle_pos}")
        
        X = np.array(X_processed).T
        y = df['label'].values
        
        return X, y, feature_names
    
    def get_model_importance(self, model_name, X, y, feature_names):
        model = self.models[model_name]
        model.fit(X, y)
        
        if model_name == 'logistic':
            importance = np.abs(model.coef_[0])
        else:
            importance = model.feature_importances_
            
        return importance
    
    def get_shap_importance(self, X, y, feature_names):
        model = xgb.XGBClassifier(n_estimators=100, random_state=42)
        model.fit(X, y)
        
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        return np.abs(shap_values).mean(axis=0)
    
    def analyze_all_models(self, df, window_size):
        X, y, feature_names = self.prepare_data(df, window_size)
        
        results = {}
        # Get importance from each model
        for model_name in self.models.keys():
            importance = self.get_model_importance(model_name, X, y, feature_names)
            results[model_name] = self.aggregate_importance(importance, feature_names)
            
        # Add SHAP importance
        try:
            shap_importance = self.get_shap_importance(X, y, feature_names)
            results['shap'] = self.aggregate_importance(shap_importance, feature_names)
        except Exception as e:
            print(f"SHAP analysis failed: {e}")
            
        return results
    
    def aggregate_importance(self, importance, feature_names):
        feature_importance = {}
        for feat_type in self.continuous_features + ['ss']:
            if feat_type == 'ss':
                mask = [f.startswith('ss_') for f in feature_names]
            else:
                mask = [f.startswith(feat_type + '_') for f in feature_names]
            importance_values = importance[mask]
            feature_importance[feat_type] = np.mean(importance_values)
        return feature_importance

def print_model_importance_tables(all_results):
    """Print feature importance table for each model"""
    model_names = list(all_results[0].keys())  # Get all model names
    window_sizes = list(all_results.keys())    # Get all window sizes
    
    for model in model_names:
        print(f"\n=== {model.upper()} ===")
        print("-" * 80)
        
        # Print header with window sizes
        print(f"{'Feature':<15} ", end="")
        for size in window_sizes:
            print(f"Win±{size:<12} ", end="")
        print("\n" + "-" * 80)
        
        # Get all features for this model
        features = list(all_results[0][model].keys())
        
        # Print each feature's importance across window sizes
        for feature in sorted(features):
            print(f"{feature:<15} ", end="")
            for size in window_sizes:
                importance = all_results[size][model].get(feature, 0)
                print(f"{importance:,.4f}    ", end="")
            print()
        print("-" * 80)

def print_window_importance_tables(all_results):
    """Print feature importance table for each window size"""
    for size in all_results.keys():
        print(f"\n=== Window Size ±{size} ===")
        print("-" * 80)
        
        # Print header
        print(f"{'Feature':<15} ", end="")
        model_names = list(all_results[size].keys())
        for model in model_names:
            print(f"{model.upper():<15} ", end="")
        print("\n" + "-" * 80)
        
        # Get all features
        features = list(all_results[size][list(all_results[size].keys())[0]].keys())
        
        # Print each feature's importance across models
        for feature in sorted(features):
            print(f"{feature:<15} ", end="")
            for model in model_names:
                importance = all_results[size][model].get(feature, 0)
                print(f"{importance:,.4f}       ", end="")
            print()
        print("-" * 80)

def analyze_features(train_df_path):
    # Define features
    continuous_features = [
        'phi', 'psi', 'omega', 'tau', 
        'chi1', 'chi2', 'chi3', 'chi4',
        'sasa', 'plDDT'
    ]
    discrete_features = ['ss']
    
    # Initialize analyzer
    analyzer = FeatureImportanceAnalyzer(continuous_features, discrete_features)
    
    # Read data
    train_df_path = os.path.abspath(os.path.expanduser(train_df_path))
    print(f"Reading data from: {train_df_path}")
    train_df = pd.read_csv(train_df_path)
    
    # Process features
    processed_df = pd.DataFrame()
    processed_df['label'] = train_df['label']
    
    def process_feature(df, feature_name):
        if feature_name == 'ss':
            return df[feature_name].apply(list)
        else:
            return df[feature_name].apply(lambda x: 
                np.array([float(v.strip()) for v in x.strip('[]').split(',')]))
    
    for feature in continuous_features:
        processed_df[feature] = process_feature(train_df, feature)
    processed_df['ss'] = train_df['ss'].apply(list)
    
    # Analyze with different window sizes
    window_sizes = [0, 1, 2, 4, 8, 16]
    all_results = {}
    
    for size in window_sizes:
        print(f"Processing window size ±{size}")
        results = analyzer.analyze_all_models(processed_df, size)
        all_results[size] = results
    
    # Print both views of feature importance
    print("\n=== ANALYSIS BY MODEL ===")
    print_model_importance_tables(all_results)
    
    print("\n=== ANALYSIS BY WINDOW SIZE ===")
    print_window_importance_tables(all_results)
    
    # Create results directory
    results_dir = 'feature_importance_results'
    os.makedirs(results_dir, exist_ok=True)
    
    # Create comparison visualizations
    model_names = list(analyzer.models.keys()) + ['shap']
    
    for model_name in model_names:
        try:
            results_df = pd.DataFrame({
                size: results[model_name] 
                for size, results in all_results.items()
            })
            results_df.columns = [f'±{size}' for size in window_sizes]
            
            plt.figure(figsize=(12, 8))
            sns.heatmap(results_df, annot=True, fmt='.3f', cmap='YlOrRd')
            plt.title(f'Feature Importance vs Window Size ({model_name.upper()})')
            plt.xlabel('Window Size')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig(os.path.join(results_dir, f'feature_importance_heatmap_{model_name}.png'))
            plt.close()
            
            # Save numerical results
            results_df.to_csv(os.path.join(results_dir, f'feature_importance_{model_name}.csv'))
            
        except KeyError as e:
            print(f"Skipping {model_name} visualization due to missing results")
            continue
    
    return all_results

# Usage
if __name__ == "__main__":
    results = analyze_features('../../data/train/structure/processed_features_train.csv')

Reading data from: /home/ubuntu/data/hai/thesis/processed_features_fixed_train.csv
Processing window size ±0
[LightGBM] [Info] Number of positive: 4592, number of negative: 4261
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000749 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2556
[LightGBM] [Info] Number of data points in the train set: 8853, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518694 -> initscore=0.074812
[LightGBM] [Info] Start training from score 0.074812
Processing window size ±1
[LightGBM] [Info] Number of positive: 4592, number of negative: 4261
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001910 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7662
[LightGBM] [Info] Number of data points in the train set: 8853, number of used features: 39
[LightGBM] [Info] [